In [7]:
import pandas as pd
from google.cloud import bigquery
import nltk
import numpy as np
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import re
import string
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/emma_tebbe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/emma_tebbe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/emma_tebbe/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
bqclient = bigquery.Client()
query = (
    """
    SELECT * FROM `w266-313317.final_project.tag_asset_count`
    """
)
result = bqclient.query(query).to_dataframe()

In [3]:
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop_words = set(stopwords.words('english'))

In [4]:
result_np = result[['tag','language','auto_generated','organization_id','asset_count']].to_numpy()

In [10]:
def word_preprocessing(word):
    lower = word.lower()
    rem_punct = lower.translate(str.maketrans('', '', string.punctuation))
    lemma = [lemmatizer.lemmatize(w) for w in nltk.word_tokenize(rem_punct)]
    rem_stop = [w for w in lemma if not w in stop_words]
    rem_digits = [re.sub('\d', '<dig>', i) for i in rem_stop]
    return rem_digits
lemma_test = [word_preprocessing(i[0]) for i in result_np]
lemma_test

[['<dig><dig><dig><dig><dig><dig><dig><dig><dig>',
  'john',
  'shearer',
  'cmt',
  'award',
  'bicentennial'],
 ['episodic'],
 ['season', '<dig><dig>'],
 ['<dig><dig><dig><dig><dig><dig><dig><dig><dig>'],
 ['<dig><dig><dig><dig><dig><dig><dig><dig><dig>'],
 ['starter', 'unit'],
 ['disney', 'art', 'animation'],
 ['<dig><dig><dig><dig><dig><dig><dig><dig><dig>'],
 ['<dig><dig><dig><dig><dig><dig>aestheticsbiomedicalpa'],
 ['group'],
 ['<dig><dig><dig><dig><dig><dig><dig><dig><dig>'],
 ['tray'],
 ['insulated', 'transporter'],
 ['select'],
 ['<dig><dig><dig><dig><dig><dig><dig><dig><dig>'],
 ['nbcu', 'photo', 'bank'],
 ['<dig>', 'shelf'],
 ['<dig><dig><dig><dig><dig><dig><dig><dig><dig>'],
 ['<dig><dig><dig><dig><dig><dig><dig><dig><dig>'],
 ['colander', 'food', 'pan'],
 ['<dig><dig><dig><dig>'],
 ['color', 'image'],
 ['<dig><dig><dig><dig><dig><dig><dig><dig>',
  'cmt',
  'award',
  'fan',
  'viewing',
  'partykempin'],
 ['dust', 'extractor'],
 [],
 ['<dig><dig><dig><dig><dig><dig><dig>